In [39]:
import pandas as pd
import ast

# Se lee de cada línea del dataset
filas_items = []
with open('Jsons/Australian_user_reviews.json', encoding='utf-8') as f:
    for line in f.readlines():
        filas_items.append(ast.literal_eval(line))

# Se convierte en dataframe
df_reviews = pd.DataFrame(filas_items)

# Guarda el DataFrame en un archivo CSV en la carpeta "Archivos CSV"
df_reviews.to_csv('CSV/Australian_user_reviews.csv', index=False)

In [2]:
# para ver que tiene el archivo y si cuenta con nulos.
import pandas as pd

df_reviews = pd.read_csv('CSV/Australian_user_reviews.csv')

df_reviews.info()
df_reviews.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25485 entries, 0 to 25484
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25485 non-null  object
 1   user_url  25485 non-null  object
 2   reviews   25485 non-null  object
dtypes: object(3)
memory usage: 597.4+ KB


,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [41]:
# Observamos que el archivo no tiene nulos, procedemos a ver la composicion de la columna reviews.

import pandas as pd

# Carga el archivo CSV en un DataFrame
df_reviews = pd.read_csv('CSV/Australian_user_reviews.csv')

# Reemplaza 'nombre_columna' con el nombre de la columna que deseas explorar
column = df_reviews['reviews'][2]

# Muestra la estructura de la columna
print(column)

[{'funny': '', 'posted': 'Posted February 3.', 'last_edited': '', 'item_id': '248820', 'helpful': 'No ratings yet', 'recommend': True, 'review': "A suitably punishing roguelike platformer.  Winning feels good.  Progressive unlocks mean a good slog ending in failure doesn't feel like a waste."}, {'funny': '', 'posted': 'Posted December 4, 2015.', 'last_edited': 'Last edited December 5, 2015.', 'item_id': '370360', 'helpful': 'No ratings yet', 'recommend': True, 'review': '"Run for fun? What the hell kind of fun is that?"'}, {'funny': '', 'posted': 'Posted November 3, 2014.', 'last_edited': '', 'item_id': '237930', 'helpful': 'No ratings yet', 'recommend': True, 'review': 'Elegant integration of gameplay, story, world development and aesthetic.'}, {'funny': '', 'posted': 'Posted October 15, 2014.', 'last_edited': '', 'item_id': '263360', 'helpful': 'No ratings yet', 'recommend': True, 'review': 'Random drops and random quests, with stat points.  Animation style reminiscent of the era bef

In [42]:
# Reviso si hay duplicados por user_id

hay_duplicados = df_reviews['user_id'].duplicated().any()
print(hay_duplicados)

True


In [43]:
# Para eliminar duplicados por user_id

df_reviews = pd.read_csv('CSV/Australian_user_reviews.csv')

# Elimina los duplicados por "user_id"
df_reviews = df_reviews.drop_duplicates(subset=['user_id'])

# Guarda el DataFrame modificado en el mismo archivo CSV
df_reviews.to_csv('CSV/Australian_user_reviews.csv', index=False)

In [44]:
# Compruebo nuevo estado del archivo.
# se paso de 25799 a 25485

import pandas as pd

df_reviews = pd.read_csv('CSV/Australian_user_reviews.csv')

df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25485 entries, 0 to 25484
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25485 non-null  object
 1   user_url  25485 non-null  object
 2   reviews   25485 non-null  object
dtypes: object(3)
memory usage: 597.4+ KB


In [14]:
# Procedemos a desanidar la informacion de la columna reviews.

import pandas as pd
import ast

# Lee el archivo JSON en una lista de diccionarios
filas_reviews = []
with open('Jsons/Australian_user_reviews.json', 'r', encoding='utf-8') as archivo_json:
    for linea in archivo_json:
        objeto_json = ast.literal_eval(linea)
        filas_reviews.append(objeto_json)

# Normaliza la columna 'reviews' y selecciona los metadatos
df_reviews = pd.json_normalize(filas_reviews, record_path=['reviews'], meta=['user_id', 'user_url'])

# Guarda el DataFrame en un archivo CSV
df_reviews.to_csv('CSV/Australian_user_reviews_normalizada.csv', index=False)

In [15]:
# Se valida el resultado.
import pandas as pd

df_normalizada_r = pd.read_csv('CSV/Australian_user_reviews_normalizada.csv')
df_normalizada_r.head(3)

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url
0,NaN,"Posted November 5, 2011.",NaN,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,NaN,"Posted July 15, 2011.",NaN,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,NaN,"Posted April 21, 2011.",NaN,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,http://steamcommunity.com/profiles/76561197970...


In [16]:
# para saber si los user_id tienen duplicados en los item_id
# Se valida ya que por la funcionalidad de steam solo se puede reseñar 1 vez el juego.
import pandas as pd

# Carga el DataFrame a partir del archivo CSV
df_normalizada_r = pd.read_csv('CSV/Australian_user_reviews_normalizada.csv')

# Agrupa por 'user_id' y 'item_id' y cuenta las ocurrencias
duplicados = df_normalizada_r.groupby(['user_id', 'item_id']).size().reset_index(name='count')

# Filtra las filas donde count sea mayor que 1, lo que significa que hay duplicados
duplicados = duplicados[duplicados['count'] > 1]

if duplicados.empty:
    print('No se encontraron duplicados en user_id e item_id.')
else:
    print('Se encontraron duplicados en user_id e item_id:')
    print(duplicados)

Se encontraron duplicados en user_id e item_id:
                     user_id  item_id  count
91                  05041129      440      2
92                  05041129      730      2
93                  05041129   277430      2
184    111222333444555666888      730      2
185    111222333444555666888     8870      2
...                      ...      ...    ...
57757         xXAussieRockXx   220240      2
57758         xXAussieRockXx   304050      2
58019            yolofaceguy   250320      2
58020            yolofaceguy   261030      2
58282              zeroblade   219640      2

[862 rows x 3 columns]


In [18]:
# Se implementa este codigo para ver si los datos de los user_id con item_id duplicados.
user_id_to_preview = '05041129'

# Filtra las filas que coincidan con el user_id específico
filtered_rows = df_normalizada_r[df_normalizada_r['user_id'] == user_id_to_preview]

# Genera una vista previa de las primeras 5 filas para ese user_id
preview = filtered_rows.head(20)

# Imprime la vista previa en la consola
print(preview)

                                  funny                   posted last_edited  \
13410                               NaN     Posted May 18, 2015.         NaN   
13411  1 person found this review funny  Posted January 5, 2015.         NaN   
13412                               NaN     Posted May 23, 2015.         NaN   
31984                               NaN     Posted May 18, 2015.         NaN   
31985  1 person found this review funny  Posted January 5, 2015.         NaN   
31986                               NaN     Posted May 23, 2015.         NaN   

       item_id                                         helpful  recommend  \
13410      730  4 of 4 people (100%) found this review helpful       True   
13411   277430  1 of 1 people (100%) found this review helpful       True   
13412      440   1 of 2 people (50%) found this review helpful      False   
31984      730  4 of 4 people (100%) found this review helpful       True   
31985   277430  1 of 1 people (100%) found this review

In [2]:
# para eliminar los iten_id duplicados en los user_id.
import pandas as pd

# Carga el DataFrame desde el archivo CSV
df_normalizada_r = pd.read_csv('CSV/Australian_user_reviews_normalizada.csv')

# Elimina las filas duplicadas
df_normalizada_r = df_normalizada_r.drop_duplicates(subset=['user_id', 'item_id'])

# Sobrescribe el archivo CSV original
df_normalizada_r.to_csv('CSV/Australian_user_reviews_normalizada.csv', index=False)

In [6]:
# se realiza limpieza manual de los items_id que quedaron corridos o vacios.
# se usa este codigo para eliminar todas la columnas que no pertenezcan al archivo.

import pandas as pd

# Lee el archivo CSV en un DataFrame
df_normalizada_r = pd.read_csv('CSV/Australian_user_reviews_normalizada.csv')

# Lista de columnas que deseas conservar
columnas_deseadas = ['funny', 'posted', 'last_edited', 'item_id', 'helpful', 'recommend', 'review', 'user_id', 'user_url']

# Eliminar las columnas no deseadas
df_normalizada_r = df_normalizada_r[columnas_deseadas]

# Sobrescribe el archivo original con las columnas seleccionadas
df_normalizada_r.to_csv('CSV/Australian_user_reviews_normalizada.csv', index=False)

C:\Users\Alvaro Javier\AppData\Local\Temp\ipykernel_12508\3933219808.py:6: DtypeWarning: Columns (9,10,12,15,18,20,22,26,29,32,35,38,40,43,45,47,50,52,55,57,59,61,63,65,68,71,74,76,78,80,82,84,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,2

In [3]:

df_normalizada_r = pd.read_csv('CSV/Australian_user_reviews_normalizada.csv')
df_normalizada_r.head(1)

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url
0,NaN,"Posted November 5, 2011.",NaN,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...


In [ ]:
# Despues de analizar el funcionamiento de las reviews en steam, nos dimos cuenta que:
# un juegador no puede tener mas de una review de un mismo juego, solo se puede editar.
# un jugador esta obligado a indicar si recomienda o no el juego.
# Un jugador puede desactivar la casilla de comentarios de su review.
# se procede a eliminar funny, posted, last_edited, helpful.	